In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from utils import *

from datasets import load_dataset
import pandas as pd
import random
import time
import pickle
import os

# Data

In [16]:
dataset = load_dataset('glue', 'sst2').shuffle()

Reusing dataset glue (C:\Users\Fabrice\.cache\huggingface\datasets\glue\sst2\1.0.0\7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)


# Evaluation

In [17]:
# # generate 100 test suites
# test_suites = {}
# for i in range(100):
#     n = 100
#     j = random.randint(0, len(dataset['train']['sentence']) - 1 - n)
#     X, y = dataset['train']['sentence'][j:j+n], dataset['train']['label'][j:j+n]
#     test_suite = {'data': [str(x) for x in X], 'target': y}
#     test_suites[i] = test_suite
# pkl_save(test_suites, 'assets/SST2/test_suites.pkl')
test_suites = pkl_load('assets/SST2/test_suites.pkl')

In [25]:
INV_test_suites = transform_test_suites(test_suites, num_transforms=2, task='sentiment', tran='INV')
pkl_save(INV_test_suites, 'assets/SST2/INV_test_suites.pkl')

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:35<00:00,  1.05it/s]


In [26]:
SIB_test_suites = transform_test_suites(test_suites, num_transforms=2, task='sentiment', tran='SIB')
pkl_save(SIB_test_suites, 'assets/SST2/SIB_test_suites.pkl')

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:09<00:00,  1.43it/s]


In [28]:
both_test_suites = transform_test_suites(test_suites, num_transforms=2, task='sentiment', tran=None)
pkl_save(both_test_suites, 'assets/SST2/both_test_suites.pkl')

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:59<00:00,  1.19s/it]


# Testing

In [3]:
test_suites = pkl_load('assets/SST2/test_suites.pkl')
INV_test_suites = pkl_load('assets/SST2/INV_test_suites.pkl')
SIB_test_suites = pkl_load('assets/SST2/SIB_test_suites.pkl')
both_test_suites = pkl_load('assets/SST2/both_test_suites.pkl')

In [4]:
def get_acc(y_pred, y_true):
    total = y_true.size(0)
    correct = (y_pred == y_true).sum().item()
    return correct / total

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

def load_huggingface_model(model_name, device, max_length=500):
    tokenizer = AutoTokenizer.from_pretrained(model_name,)
    model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
    def pipeline(sentence):
        encode = tokenizer(sentence, 
                           padding=True, 
                           truncation=True, 
                           max_length=max_length, 
                           return_tensors="pt").to(device)
        logits = model(**encode)[0]
        soft_m = torch.softmax(logits, dim=1)
        return soft_m
    return pipeline

MODEL_NAME = "textattack/bert-base-uncased-SST-2"
model = load_huggingface_model(MODEL_NAME, device, 250)

In [6]:
tss = {
    'test_suites' : test_suites,
    'INV_test_suites' : INV_test_suites,
    'SIB_test_suites' : SIB_test_suites,
    'both_test_suites' : both_test_suites
}

In [8]:
for t_name, test_suites in tss.items():
    print('starting {}...'.format(t_name))
    for idx, t in test_suites.items():
        logits = model([str(x) for x in t['data']])
        y_pred = torch.argmax(logits, dim=1)
        y_true = torch.tensor(t['target'])
        acc = get_acc(y_pred, y_true)
        t['performance'] = {
            'MODEL_NAME' : MODEL_NAME,
            'acc' : acc
        }
        print('test suite {} acc: {}'.format(idx, acc))
    file_path = 'assets/SST2/' + t_name + '_w_acc.pkl'
    pkl_save(test_suites, file_path)
    print('saving {}'.format(file_path))

starting test_suites...
test suite 0 acc: 0.99
test suite 1 acc: 1.0
test suite 2 acc: 1.0
test suite 3 acc: 0.99
test suite 4 acc: 0.99
test suite 5 acc: 0.98
test suite 6 acc: 0.96
test suite 7 acc: 0.97
test suite 8 acc: 0.99
test suite 9 acc: 1.0
test suite 10 acc: 0.99
test suite 11 acc: 0.99
test suite 12 acc: 0.99
test suite 13 acc: 0.99
test suite 14 acc: 1.0
test suite 15 acc: 1.0
test suite 16 acc: 0.99
test suite 17 acc: 0.99
test suite 18 acc: 0.96
test suite 19 acc: 0.98
test suite 20 acc: 0.99
test suite 21 acc: 0.99
test suite 22 acc: 0.96
test suite 23 acc: 0.98
test suite 24 acc: 0.99
test suite 25 acc: 0.97
test suite 26 acc: 0.98
test suite 27 acc: 0.98
test suite 28 acc: 0.98
test suite 29 acc: 0.96
test suite 30 acc: 0.99
test suite 31 acc: 1.0
test suite 32 acc: 0.99
test suite 33 acc: 1.0
test suite 34 acc: 0.99
test suite 35 acc: 0.98
test suite 36 acc: 0.96
test suite 37 acc: 0.97
test suite 38 acc: 0.98
test suite 39 acc: 1.0
test suite 40 acc: 0.98
test suite

test suite 36 acc: 0.64
test suite 37 acc: 0.68
test suite 38 acc: 0.65
test suite 39 acc: 0.58
test suite 40 acc: 0.75
test suite 41 acc: 0.58
test suite 42 acc: 0.69
test suite 43 acc: 0.64
test suite 44 acc: 0.61
test suite 45 acc: 0.68
test suite 46 acc: 0.67
test suite 47 acc: 0.61
test suite 48 acc: 0.63
test suite 49 acc: 0.72
test suite 50 acc: 0.57
test suite 51 acc: 0.64
test suite 52 acc: 0.54
test suite 53 acc: 0.62
test suite 54 acc: 0.64
test suite 55 acc: 0.65
test suite 56 acc: 0.58
test suite 57 acc: 0.67
test suite 58 acc: 0.58
test suite 59 acc: 0.65
test suite 60 acc: 0.71
test suite 61 acc: 0.69
test suite 62 acc: 0.64
test suite 63 acc: 0.65
test suite 64 acc: 0.59
test suite 65 acc: 0.68
test suite 66 acc: 0.63
test suite 67 acc: 0.64
test suite 68 acc: 0.66
test suite 69 acc: 0.57
test suite 70 acc: 0.67
test suite 71 acc: 0.59
test suite 72 acc: 0.59
test suite 73 acc: 0.63
test suite 74 acc: 0.58
test suite 75 acc: 0.6
test suite 76 acc: 0.64
test suite 77 acc

In [9]:
# BERT
for t_name, test_suites in tss.items():
    count = 0
    total = 0
    for idx, t in test_suites.items():
        total += t['performance']['acc']
        count += 1
    avg_acc = total / count
    print('average acc: {0:1.2f} | {1}'.format(avg_acc, t_name))

average acc: 0.99 | test_suites
average acc: 0.79 | INV_test_suites
average acc: 0.60 | SIB_test_suites
average acc: 0.64 | both_test_suites


# Sample some transforms to see if they're reasonable

In [ ]:
import csv

In [8]:
filename = "./assets/csv_examples/sst2_inv_examples.csv"
    
# writing to csv file  
with open(filename, 'w') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)  
    csvwriter.writerow(['text', 'label', 'sound good?'])
    # writing the fields  
    for i in range(50):
        row = [INV_test_suites[i]['data'][i], INV_test_suites[i]['target'][i], True ]#, INV_test_suites[i]['ts'][i] ]
        csvwriter.writerow(row)

In [9]:
filename = "./assets/csv_examples/sst2_sib_examples.csv"
    
# writing to csv file  
with open(filename, 'w') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)  
    csvwriter.writerow(['text', 'label', 'sound good?'])
    # writing the fields  
    for i in range(50):
        row = [SIB_test_suites[i]['data'][i], SIB_test_suites[i]['target'][i], True ]#, INV_test_suites[i]['ts'][i] ]
        csvwriter.writerow(row)